<a href="https://colab.research.google.com/github/marriamaslova/compling_nlp_hse_course/blob/master/notebooks/peft/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1.

Дообучите языковую модель на датасете инструкций, используя LoRA. Проверьте, что дообученная модель отличается от изначальной - сгенерируйте продолжения для одних и тех же промптов и сравните результаты.

Вы можете взять за основу код семинара PEFT, изменив датасет цитат на датасет инструкций (можно просто скопировать из семинара про General_instruct_fine-tuning). 
Можно использовать alpaca_dataset, датасет Dolly 2 или ваш собственный переведенный датасет (или все вместе). 
Важно использовать модель с большим количеством параметров (относительно семинара по General instruct fine-tuning). Например, можно попробовать OPT-1.3b,OPT-2.7b или ai-forever/rugpt3large_based_on_gpt2.



In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b", 
    load_in_8bit=True, 
    device_map='auto',
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

In [ ]:
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=32,
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

Загрузила датасет инструкций alpaca на Hugging Face, так с большей вероятностью не возникнет проблем с данными. 

In [ ]:
import transformers
from datasets import load_dataset

data = load_dataset("marriamaslova/instructions_for_peft")
data = data.map(lambda samples: tokenizer(samples['instruction']), batched=True)
data = data.map(lambda samples: tokenizer(samples['input']), batched=True)
data = data.map(lambda samples: tokenizer(samples['output']), batched=True)

trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=400, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
model.save_pretrained('opt_1.3_lora')

In [ ]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "opt_1.3_lora"

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-1.3b", 
                                             return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [ ]:
def generate(text, tokenizer, model):
  batch = tokenizer(text, return_tensors='pt').to('cuda')
  output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.0, no_repeat_ngram_size=2)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
generate("I have a dream that",  tokenizer, model)

"I have a dream that one day, the world will be a better place.\nI've got a feeling that's not going to happen."

In [ ]:
generate("Star is",  tokenizer, model)

"Star is a good guy.\nHe's a great guy, but he's not a very good actor."

In [ ]:
generate("How can we reduce air pollution?",  tokenizer, model)

'How can we reduce air pollution?\n\nAir pollution is a major problem in many parts of the world. It is estimated that more than 1.5 billion people worldwide suffer from air-pollution-related diseases.\nThe World Health Organization (WHO) estimates that air pollutants cause'

In [ ]:
generate("You are in love when",  tokenizer, model)

'You are in love when you are with someone. You are not in a relationship when your heart is not with them.\nI think this is the best answer.'

In [ ]:
model_tuned = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
generate("I have a dream that",  tokenizer, model_tuned)

'I have a dream that one day, all people will live in harmony and peace.\n\nI believe that we can achieve this dream by working together to create a better world. I believe in the power of people working towards a common goal. Together, we will create the'

In [ ]:
generate("Star is",  tokenizer, model_tuned)

'Star is a star.\n\nThe sun is the sun. \nIt is bright and shining.\n\n\nStar, star, bright star!\nYou are the brightest star in the sky.'

In [ ]:
generate("How can we reduce air pollution?",  tokenizer, model_tuned)

'How can we reduce air pollution?\n\n1. Reduce the use of fossil fuels.\n2. Use renewable energy sources. \n3. Improve public transportation. 4. Increase the efficiency of vehicles. 5. Create more green spaces. 6. Clean up the air. 7'

In [ ]:
generate("You are in love when",  tokenizer, model_tuned)

'You are in love when you feel the warmth of his arms around you. You feel his love for you and you know that you will be together forever.\n\nYou feel a sense of peace and contentment when he is by your side. He is your everything and he'

Результат понравился, на любой заданный промт дообученная модель дает более интересный ответ